In [ ]:
import pandas as pd
import requests
import json

In [ ]:
url =  'https://api.yelp.com/v3/businesses/search'

In [ ]:
from keys import client_id, api_key

In [ ]:
headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

term='sushi'
location='brooklyn'
price='1,2,3,4'
radius=20000

url_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "price" : price,
                "limit": 50,
                "radius": radius
            }

url_review_params = {
                "term": term.replace(' ', '+'),
                "location": location.replace(' ', '+'),
                "price" : price,
                "limit": 50,
                "radius": radius
            }

response = requests.get(url, headers=headers, params=url_params)

---

### Call Check

if response.status_code == 200:
    print('all good')
else:
    print(response.status_code)

---

data = response.json()
data.keys()

data

data['total']

---

## Functions

def yelp_call_businesses(url_params, api_key):
    headers = {'Authorization': 'Bearer {}'.format(api_key),}
    url =  'https://api.yelp.com/v3/businesses/search'
    response=requests.get(url, headers=headers, params=url_params)
    return response.json()

def parse_data(list_of_data):
    parsed_data = []
    columns = ['name', 'id', 'review_count', 'price', 'rating']
    
    for business in list_of_data['businesses']:
        biz_list = [business['name'],
            business['id'],
            business['review_count'], 
            business['price'],
            business['rating']]
        parsed_data.append(biz_list)
    
    df = pd.DataFrame(parsed_data)
    
    df.columns = columns
    
    return df

def data_save(data, csv_filename):
    return data.to_csv(csv_filename)
    # your code to save the current results with all of the other results. 
    # I would save the data every time you pull 50 results
    # in case something breaks in the process.

test = yelp_call_businesses(url_params, api_key)


parse_data(test)

---

## Pagination

columns = ['name', 'id', 'review_count', 'price', 'rating']

new_df = pd.DataFrame(columns=columns)

data_save(new_df, 'yelp_api_lab_2.csv')

pd.read_csv('yelp_api_lab_2.csv', index_col=0)

# Write a script that combines the three functions above into a single process.

# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
# while cur < num and cur < 1000:
while cur < 1000:
    
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    
    #make your API call with the new offset number
    results = yelp_call_businesses(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    data = parse_data(results)
    
    #open csv file
    df_former = pd.read_csv('yelp_api_lab_2.csv', index_col=0)
    
    #concatenate data on new_df
    df = pd.concat([df_former,data], axis=0, ignore_index=True)
    
    # insert your parsed results into the db
    data_save(df, 'yelp_api_lab_2.csv')
    
    #increment the counter by 50 to move on to the next results
    cur += 50

---

df = pd.read_csv('yelp_api_lab_2.csv', index_col=0)

# write Pandas code to pull back all of the business ids 
# you will need these ids to pull back the reviews for each restaurant

business_id=df['id']
business_id

# write a function that takes a business id 
# and makes a call to the API for reivews
def id_call(business_id, api_key):
    reviews_container = []
    
    headers = {'Authorization': 'Bearer {}'.format(api_key),}
    
    for i in business_id: 
        url = f'https://api.yelp.com/v3/businesses/{i}/reviews'
        response=requests.get(url, headers=headers)
        reviews_container.append(response.json())
        
    return reviews_container

# Write a function to parse out the relevant information from the reviews
def parse_data_reviews(list_of_data):
    parsed_data = []
    columns = ['rating', 'text', 'time_created']
    
    for review in list_of_data:
        for i in review['reviews']:
            review_list = [i['rating'], 
                i['text'],
                i['time_created']]
            parsed_data.append(review_list)
    
    df = pd.DataFrame(parsed_data)
    
    df.columns = columns
    
    return df

reviews_container = id_call(business_id, api_key)

reviews_container[0]['reviews'][0]['rating']

df_reviews = parse_data_reviews(reviews_container)

# write a function to insert the parsed data into the reviews table

# combine the functions above into a single script  